<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/Schachcomputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schachcomputer


## Klassen

In [ ]:
#https://raw.githubusercontent.com/prateeksan/chesspi/development/sample_data/Kasparov.pgn
# TODO: Patt, (Rochade), En Passant, (Bauernumwandlung)
!wget -nc https://raw.githubusercontent.com/ollihansen90/Mathe-SH/refs/heads/main/utils/schach_utils.py

if False:
    schachbrett.reset()
    schachbrett["E2"] = leer
    schachbrett.male()
    moegliche_zuege = get_alle_bewegungen(schachbrett)
    print(moegliche_zuege)
    print(moegliche_zuege[randint(0,len(moegliche_zuege)-1)])

    schachbrett = Spielfeld()
    schachbrett2 = Spielfeld()
    schachbrett.male()

    schachbrett["E2"] = leer
    schachbrett["E3"] = bauer
    schachbrett.male()

## Züge

# Bewertungsfunktion

In [ ]:
leer = 0
bauer = 1

schachbrett = Spielfeld()
schachbrett.reset()
schachbrett["H7"] = leer
schachbrett["C2"] = leer
schachbrett["E2"] = leer
schachbrett["G2"] = leer
schachbrett["D2"] = leer
schachbrett["D4"] = bauer
schachbrett["F3"] = bauer
schachbrett.male()
print(berechne_wert(schachbrett))
schachbrett.male()

In [ ]:
import copy
from random import randint

def argsort(l):
    liste = l[:]
    output = []
    kleinster_wert = -1000000000000000000
    for _ in range(len(liste)):
        kleinster = 100000000000000000000000
        kleinstes_i = 0
        for i in range(len(liste)):
            if liste[i]<kleinster and liste[i]>=kleinster_wert:
                #print(liste[i], kleinster)
                kleinster = liste[i]
                kleinstes_i = i
        kleinster_wert = kleinster
        output.append(kleinstes_i)
        liste[kleinstes_i] = 100000000
    return output

def waehle_zufall(scores):
    #print(scores)
    scores = [(eintrag-min(scores)+1)**2 for eintrag in scores]
    #print(scores)
    z = randint(0,int(2*sum(scores)))
    for i in range(len(scores)):
        eintrag = scores[i]
        z -= 2*eintrag
        if z<1:
            return i

def ki_zug(schachbrett):
    zuege = get_alle_bewegungen(schachbrett)
    hilfsbrett = Spielfeld()
    werte = []
    for z in zuege:
        # hilfsbrett.feld = schachbrett.feld
        hilfsbrett.feld = copy.deepcopy(schachbrett.feld)
        zug(hilfsbrett, z)
        werte.append(berechne_wert(hilfsbrett))
    #print(zuege)
    #print(werte)
    #hilfsbrett.male()
    zuege = [zuege[i] for i in argsort(werte)][::-1]
    #print(sorted(werte))
    next_zug = zuege[waehle_zufall(sorted(werte, reverse=True)[:len(werte)//3])]
    return next_zug

def ki_zug_det(schachbrett):
    zuege = get_alle_bewegungen(schachbrett)
    hilfsbrett = Spielfeld()
    werte = []
    for z in zuege:
        hilfsbrett.feld = copy.deepcopy(schachbrett.feld)
        zug(hilfsbrett, z)
        werte.append(berechne_wert(hilfsbrett))
    zuege = [zuege[i] for i in argsort(werte)][::-1]
    #print(list(zip(zuege, sorted(werte)[::-1])))
    next_zug = zuege[0]
    return next_zug

def ki_minimax(schachbrett):
    zuege = get_alle_bewegungen(schachbrett)
    hilfsbrett = Spielfeld()
    werte = []
    for z in zuege:
        #print(z, end=" ")
        hilfsbrett.feld = copy.deepcopy(schachbrett.feld)
        zug(hilfsbrett, z)
        hilfsbrett.invertiere()
        zug(hilfsbrett, ki_zug_det(hilfsbrett))
        hilfsbrett.invertiere()
        wert = berechne_wert(hilfsbrett)
        werte.append(wert)
    zuege = [zuege[i] for i in argsort(werte)][::-1]
    next_zug = zuege[0]
    return next_zug


schachbrett = Spielfeld()
schachbrett.reset()
schachbrett.male()
z = ki_minimax(schachbrett)
zug(schachbrett, z)
schachbrett.male()
berechne_wert(schachbrett)

In [ ]:
liste1 = [40,7,5,6,20,40,0,1,25,8]
    #    [ 0,1,2,3, 4, 5,6,7, 8,9]
#print(sorted(liste1))
gesucht = [0,1,5,6,4,8,7,2,3]
def argsort(l):
    liste = l[:]
    output = []
    kleinster_wert = -1000000000000000000
    for _ in range(len(liste)):
        kleinster = 100000000000000000000000
        kleinstes_i = 0
        for i in range(len(liste)):
            if liste[i]<kleinster and liste[i]>=kleinster_wert:
                #print(liste[i], kleinster)
                kleinster = liste[i]
                kleinstes_i = i
        kleinster_wert = kleinster
        output.append(kleinstes_i)
        liste[kleinstes_i] = 100000000
    return output
srt_idc = argsort(liste1)
print(liste1)
print(srt_idc)
print([liste1[i] for i in srt_idc])
print(sorted(list(range(len(liste1))), key=lambda x: liste1[x]))

In [ ]:
schachbrett = Spielfeld()
schachbrett.reset()
schachbrett.male()

while True:
    # Zug Weiß
    print(get_alle_bewegungen(schachbrett))
    print(berechne_wert(schachbrett))
    z = input("Weiß gibt einen Zug ein: ")
    while True:
        if zug(schachbrett, z) == None:
            break
        z = input("Weiß gibt einen Zug ein: ")
    schachbrett.male()

    # Zug Schwarz
    schachbrett.invertiere()
    if False:
        z = input("Schwarz gibt einen Zug ein: ")
        z = flip_pos(z)
        while True:
            if zug(schachbrett, z) == None:
                break
            z = input("Schwarz gibt einen Zug ein: ")
            z = flip_pos(z)

    z = ki_minimax(schachbrett)
    zug(schachbrett, z)
    schachbrett.invertiere()
    schachbrett.male()

    # König weg?
    schwarzer_koenig = False
    weisser_koenig = False
    for i in range(8):
        for j in range(8):
            if abs(schachbrett.feld[i][j])==koenig:
                if schachbrett.feld[i][j]>0:
                    weisser_koenig = True
                else:
                    schwarzer_koenig = True
    if weisser_koenig and not schwarzer_koenig:
        print("Weiß hat gewonnen!")
        break
    elif schwarzer_koenig and not weisser_koenig:
        print("Schwarz hat gewonnen!")
        break


In [ ]:
test = [-3,-2,-1,0,1,2,3,2,6,43,2,5,4,2,1,2,7,8,9,7,5]
n = len(test)//2
test = sorted(test, reverse=True)[:n]

def waehle_zufall(scores):
    #print(scores)
    scores = [eintrag-min(scores)+1 for eintrag in scores]
    #print(scores)
    z = randint(0,sum(scores))
    for i in range(len(scores)):
        eintrag = scores[i]
        z -= eintrag
        if z<1:
            return i

liste = 10*[0]
for i in range(1000):
    liste[waehle_zufall(test)] += 1

print([i/max(test) for i in test])
print([i/max(liste) for i in liste])
print(liste)

In [ ]:
!wget https://raw.githubusercontent.com/prateeksan/chesspi/development/sample_data/Kasparov.pgn -O "Kasparov.txt"

In [ ]:
with open("Kasparov.txt", "rb") as file:
    zeilen = file.readlines()


In [ ]:
spiele = ("\n".join([z.decode('ascii').strip() for z in zeilen[:34000]])).split("\n\n")[1::2]
print(len(spiele))


In [ ]:
print(spiele[0])

In [ ]:
spielliste = []

for spiel in spiele:
    spiel = spiel.replace("\n", " ")
    spiel_ohne_ergebnis = spiel.split("  ")[0]+"  "
    zugliste = [z.split(" ")[:2] for z in spiel_ohne_ergebnis.split(".")[1:]]
    spielliste.append(zugliste)

In [ ]:
print(*spielliste[0], sep="\n")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

feld = np.zeros([8,8])
pos = get_index("B1")
feld[pos[0], pos[1]] += 1
pos = get_index("G1")
feld[pos[0], pos[1]] += 1
pos = get_index("B8")
feld[pos[0], pos[1]] += 1
pos = get_index("G8")
feld[pos[0], pos[1]] += 1

print(feld)
for zugliste in spielliste:
    for zug in zugliste:
        for z in zug:
            z = z.replace("+", "")
            if len(z)>0 and z[0]=="K":
                pos = get_index(z[-2:])
                feld[pos[0], pos[1]] += 1

plt.figure()
plt.imshow(feld)
plt.colorbar()
plt.show()
